In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_excel('classeur2_version.xls', na_values=float('nan')) # can also index sheet by name or fetch all sheets
target = df.loc[df["nombre employés"] == "none"]
companies = target.ENTREPRISE

In [3]:
companies.head()

1927          gondrand
1928      auer-gianola
1929           cavamac
1930             curie
1931                eu
1932          gondrand
1933         hansgrohe
1934       legouessant
1935       maisonbleue
1936    r-chevalier-sa
1937            sinnae
1938     squarehabitat
1939          rdmgroup
1940                fr
1941          prolians
1942                fr
1943            altrad
1944         castorama
1945       labogilbert
1946             marie
Name: ENTREPRISE, dtype: object

In [4]:
 driver = webdriver.Chrome("C:/Users/eliot/Documents/scraping/chromedriver.exe")

In [15]:
def findNumberOfEmployees(wordlist):
    flaglist=["salariés", "collaborateurs", "employés", "personnes"]
    for i in range(0,len(wordlist)):
        if wordlist[i] in flaglist:
            if i>=2 and wordlist[i-1].isnumeric() and wordlist[i-2].isnumeric():
                return "".join([wordlist[i-2], wordlist[i-1]])

            elif wordlist[i-1].isnumeric:
                return wordlist[i-1]

            elif i<len(wordlist)-1 and (wordlist[i+1]== ":") :
                if i<len(wordlist)-2 and wordlist[i+1].isnumeric() and wordlist[i+2].isnumeric():
                    return "".join([wordlist[i+1], wordlist[i+2]])
                
                elif wordlist[i+1].isnumeric:
                    return wordlist[i+1]
                else:
                    return "none"
            else:
                return "none"

def findNumberOfEmployeesInTable(divList):
    tdList = divList.find('table').find_all('td')
    flagEmployeesWords=["effectif", "effectifs", "employé","employés"]
    for i in range(0,len(tdList)):
        if any(word in tdList[i].text.lower() for word in flagEmployeesWords ):
            return(tdList[i+1].text)
        else:
            return "none"

In [ ]:
import time
import random
import string

#Values found by inspecting google's SERP
regexKnowledgeCard = re.compile('Z0LcW XcVN5d')
regexNLP = re.compile("LGOjhe")
regexTable = re.compile("webanswers-webanswers_table__webanswers-table") 

employees_number=[] #List to store the number of employees 
companies_name=[] #List to store the companies' name 

for company in companies:
    print('-----', company,'-----')
    infoNotFound = True

    if isinstance(company, str) and (company not in ["gmail", "hotmail", "outlook", "wanadoo","orange", "sfr", "live"]): # valid company name
        infoNotFound = False
        print('valid company name')
        if company in companies_name : #company already scraped
            print('company already scraped')
            index = companies_name.index(company)
            value = employees_number[index]
            employees_number.append(value if value else "none")  
            companies_name.append(company)
        else:
            time.sleep(random.uniform(1,6)) #to be detected by google a little later
            driver.get("https://www.google.com/search?q=nombre+d%27employés+"+company)
            print("https://www.google.com/search?q=nombre+d%27employés+"+company)
            content = driver.page_source
            soup = BeautifulSoup(content)
            divList=soup.findAll('div', attrs={'class':regexKnowledgeCard})
            infoNotFound = (len(divList) == 0)   

            if not infoNotFound: #knowledge card div found
                print("-> trying knowledge card...")
                for div in divList:
                    company_number = (div.text).replace(" ","").replace(" ","") if div.text else "/" #replace odd space chars
                    if company_number :
                        print("done")
                        employees_number.append(company_number)
                        companies_name.append(company)
                    else: # info not found
                        infoNotFound = True  

            if infoNotFound: #try NLP method if no knowledge card provided
                print("-> trying text description analysis...")
                divList=soup.findAll('div', attrs={'class':"LGOjhe"})
                infoNotFound = (len(divList) == 0)    
                if not infoNotFound: #text description found
                    for div in divList:
                        company_description = (div.text).translate(str.maketrans('', '', string.punctuation))
                        company_description = company_description.split(" ") if div.text else "/"
                        print(company_description)
                        company_number = findNumberOfEmployees(company_description)
                        if company_number != "none": #if a value has been found
                            employees_number.append(company_number)
                            companies_name.append(company)
                            print("done")
                        else:
                            infoNotFound = True

            if infoNotFound: #try to search in table
                print("-> trying table analysis...")
                divList=soup.find('div', attrs={'class':regexTable})
                if divList:
                    company_number = findNumberOfEmployeesInTable(divList)
                    if company_number != "none": #if a value has been found
                        employees_number.append(company_number)
                        companies_name.append(company)
                        infoNotFound = False
                        print("done")
                    else:
                        infoNotFound = True

    if infoNotFound:
        print("info not found :(")
        employees_number.append("none")
        companies_name.append(company)



In [18]:
df = pd.DataFrame({'Company name':companies_name,'#employees':employees_number}) 
df.to_csv('result.csv', index=False, encoding='utf-8')